In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

In [2]:
df_train = pd.read_csv("./dataset/train.csv")

In [3]:
df_train['hotel'] = df_train['hotel'].replace(['Resort Hotel', 'City Hotel'], [0,1])
df_train.drop(['reservation_status', 'reservation_status_date', 'ID', 'arrival_date_year', 'arrival_date_month', \
               'arrival_date_day_of_month'], axis=1, inplace=True)

In [4]:
df_train.loc[(df_train['country'] != 'PRT'), 'country'] = 'International'
df_train['country'] = df_train['country'].replace(['International', 'PRT'], [0,1])

In [5]:
df_train['agent']= df_train['agent'].fillna(0.0)
df_train['company']= df_train['company'].fillna(0.0)
df_train['children'] =  df_train['children'].fillna(0)

In [6]:
df_train

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,0,0,342,27,0,0,2,0.0,0,BB,1,Direct,Direct,0,0,0,C,C,3,No Deposit,0.0,0.0,0,Transient,-6.305161,0,0
1,1,0,257,27,0,2,1,0.0,0,HB,1,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6.0,0.0,0,Transient,75.052227,0,0
2,1,0,257,27,0,2,2,0.0,0,HB,1,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,0.0,0,Transient,74.546401,0,0
3,1,0,257,27,0,2,2,0.0,0,HB,1,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,0.0,0,Transient,76.376288,0,0
4,1,0,257,27,0,2,2,0.0,0,HB,1,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,0.0,0,Transient,49.411647,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,0,1,19,13,0,2,2,2.0,0,BB,0,Direct,Direct,0,0,0,C,C,0,No Deposit,250.0,0.0,0,Transient,79.223571,0,1
91527,0,0,28,13,0,2,2,0.0,0,BB,1,Complementary,Direct,0,0,0,A,A,0,No Deposit,0.0,0.0,0,Transient,-6.822102,0,0
91528,0,0,2,13,0,1,2,0.0,0,BB,0,Online TA,TA/TO,0,0,0,E,F,1,No Deposit,0.0,0.0,0,Transient,90.814554,0,2
91529,0,0,30,13,3,7,2,0.0,0,BB,0,Direct,Direct,0,0,0,A,A,0,No Deposit,250.0,0.0,0,Transient,38.135565,0,1


In [7]:
df_train = pd.get_dummies(df_train)

In [8]:
df_train.drop(['market_segment_Undefined', 'distribution_channel_Undefined', 'assigned_room_type_L', 'reserved_room_type_L'], \
              axis=1, inplace=True)

In [9]:
df_train

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0,0,342,27,0,0,2,0.0,0,1,0,0,0,3,0.0,0.0,0,-6.305161,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1,0,257,27,0,2,1,0.0,0,1,0,0,0,1,6.0,0.0,0,75.052227,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,1,0,257,27,0,2,2,0.0,0,1,0,0,0,0,6.0,0.0,0,74.546401,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,1,0,257,27,0,2,2,0.0,0,1,0,0,0,0,6.0,0.0,0,76.376288,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1,0,257,27,0,2,2,0.0,0,1,0,0,0,0,6.0,0.0,0,49.411647,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,0,1,19,13,0,2,2,2.0,0,0,0,0,0,0,250.0,0.0,0,79.223571,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
91527,0,0,28,13,0,2,2,0.0,0,1,0,0,0,0,0.0,0.0,0,-6.822102,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
91528,0,0,2,13,0,1,2,0.0,0,0,0,0,0,1,0.0,0.0,0,90.814554,0,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
91529,0,0,30,13,3,7,2,0.0,0,0,0,0,0,0,250.0,0.0,0,38.135565,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [10]:
df_adr = df_train['adr']
df_is_canceled = df_train['is_canceled']
df_train.drop(['adr', 'is_canceled'], axis=1, inplace=True)

# is_canceled

In [11]:
train_sz = 70389 # 最後5個月為validation (大約75:25)

X_train = df_train.iloc[:train_sz, :]
X_valid = df_train.iloc[train_sz:, :]
y_train = df_is_canceled.iloc[:train_sz]
y_valid = df_is_canceled.iloc[train_sz:]

In [12]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
y_pred_train = clf.predict(X_train)
y_pred_valid = clf.predict(X_valid)
print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_valid, y_pred_valid))

0.9165068405574734
0.8147289754990067


In [14]:
# random split
X_train, X_valid, y_train, y_valid = train_test_split(df_train, df_is_canceled, test_size=0.25, random_state=42)

In [15]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [16]:
y_pred_train = clf.predict(X_train)
y_pred_valid = clf.predict(X_valid)
print(accuracy_score(y_train, y_pred_train))
print(accuracy_score(y_valid, y_pred_valid))

0.9060861204987763
0.8896560765633876


# adr

In [17]:
train_sz = 70389 # 最後5個月為validation (大約75:25)

X_train = df_train.iloc[:train_sz, :]
X_valid = df_train.iloc[train_sz:, :]
y_train = df_adr.iloc[:train_sz]
y_valid = df_adr.iloc[train_sz:]

In [18]:
rg = xgb.XGBRegressor()
rg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [19]:
predict_adr_train = rg.predict(X_train)
predict_adr_valid = rg.predict(X_valid)

print("train rmse is: ", np.sqrt(mean_squared_error(predict_adr_train, y_train)))
print("test rmse is: ", np.sqrt(mean_squared_error(predict_adr_valid, y_valid)))

train rmse is:  18.280077530766192
test rmse is:  77.61659465192989


In [20]:
# random split
X_train, X_valid, y_train, y_valid = train_test_split(df_train, df_adr, test_size=0.25, random_state=42)

In [21]:
rg = xgb.XGBRegressor(colsample_bytree=0.5, learning_rate=0.1, max_depth=7, n_estimators=500, objective='reg:squarederror',\
                      subsample=0.7)
rg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=7,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
predict_adr_train = rg.predict(X_train)
predict_adr_valid = rg.predict(X_valid)

print("train rmse is: ", np.sqrt(mean_squared_error(predict_adr_train, y_train)))
print("test rmse is: ", np.sqrt(mean_squared_error(predict_adr_valid, y_valid)))

train rmse is:  15.505061874590817
test rmse is:  18.926053290546257


# generate test data

In [24]:
df_test = pd.read_csv("./dataset/test.csv")

In [25]:
df_test['hotel'] = df_test['hotel'].replace(['Resort Hotel', 'City Hotel'], [0,1])
df_test.drop(['ID', 'arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month'], axis=1, inplace=True)

df_test.loc[(df_test['country'] != 'PRT'), 'country'] = 'International'
df_test['country'] = df_test['country'].replace(['International', 'PRT'], [0,1])

df_test['agent']= df_test['agent'].fillna(0.0)
df_test['company']= df_test['company'].fillna(0.0)
df_test['children'] =  df_test['children'].fillna(0)

In [26]:
df_test = pd.get_dummies(df_test)

In [27]:
df_test

,hotel,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_P,assigned_room_type_A,assigned_room_type_B,assigned_room_type_C,assigned_room_type_D,assigned_room_type_E,assigned_room_type_F,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,75,13,2,5,2,0.0,0,0,0,0,0,0,9.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1,208,13,4,10,2,0.0,0,0,0,0,0,0,9.0,0.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0,12,13,2,5,2,0.0,0,1,0,0,0,0,40.0,0.0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
3,1,76,13,2,5,3,0.0,0,0,0,0,0,0,9.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1,9,13,2,4,2,2.0,0,0,0,0,0,0,9.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27854,0,108,35,2,5,2,0.0,0,0,0,0,0,0,241.0,0.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
27855,0,194,35,2,5,2,1.0,0,0,0,0,0,3,240.0,0.0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
27856,0,17,35,0,3,2,0.0,0,0,0,0,0,0,240.0,0.0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
27857,0,191,35,2,5,2,0.0,0,0,0,0,0,0,40.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [28]:
clf_final = xgb.XGBClassifier()
clf_final.fit(df_train, df_is_canceled)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
y_pred_train = clf_final.predict(df_train)

print(accuracy_score(df_is_canceled, y_pred_train))

0.9024155750510756


In [30]:
rg_final = xgb.XGBRegressor()
rg_final.fit(df_train, df_adr)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [31]:
predict_adr_train = rg_final.predict(df_train)

print("train rmse is: ", np.sqrt(mean_squared_error(predict_adr_train, df_adr)))

train rmse is:  18.532589568963193


In [33]:
is_canceled_test = clf_final.predict(df_test)
adr_test = rg_final.predict(df_test)

In [34]:
predict_is_canceled = pd.DataFrame(is_canceled_test, columns=['is_canceled'])
predict_is_canceled.to_csv('all_feature_is_canceled.csv')

In [35]:
predict_adr = pd.DataFrame(adr_test, columns=['adr'])
predict_adr.to_csv('all_feature_adr.csv')